In [7]:
import numpy as np

# Initialization for SQP
Xk = np.array([[1],[1]])
muk = np.zeros((2, 1))
W = np.identity(2)
w = np.zeros((2, 1))
k = 0
eps_term = 10e-3
counter = 0

# Function to be minimized
def func(X):
    x1 = X[0].reshape(-1, 1)
    x2 = X[1].reshape(-1, 1)
    f = x1**2 + (x2 - 3)**2
    return f

def funcx(X):
    x1 = X[0].reshape(-1, 1)
    x2 = X[1].reshape(-1, 1)
    fx = np.vstack((2 * x1, 2 * (x2 - 3)))
    return fx

def ineq_constraint(X):
    x1 = X[0].reshape(-1, 1)
    x2 = X[1].reshape(-1, 1)
    g = np.vstack((x2**2 - 2 * x1, (x2 -1)**2 + 5 * x1 - 15))
    return g

def dgx(X):
    x1 = X[0].reshape(-1, 1)
    x2 = X[1].reshape(-1, 1)
    dg_x = np.vstack((np.hstack((np.array([[-2]]), 2 * x2)), np.hstack((np.array([[5]]), 2 * (x2 -1)))))
    return dg_x

def del_lagrangian(X, mu):
    x1 = X[0].reshape(-1, 1)
    x2 = X[1].reshape(-1, 1)
    delLx = np.hstack(((2 * x1), (2 * (x2 -3)))) + np.matmul(np.transpose(mu), np.vstack((np.hstack((np.array([[-2]]), 2*x1)),np.hstack((np.array([[5]]), 2*(x2 -1))))))
    delLmu = np.hstack((x2**2 - 2 * x1, (x2 - 1)**2 + 5 * x1 - 15))
    return delLx, delLmu

def linesearch(f, df, g, dg, x, s, mu_old, w_old):
    t = 0.2
    b = 0.8
    a = 1

    D = s

    w = np.hstack((abs(mu_old), 0.5 * (w_old + abs(mu_old)))).max(1).reshape(-1,1)
    counter = 0
    while counter < 100:
        f_a = f(x + a * D) + np.matmul(np.transpose(w),abs(np.hstack((np.zeros_like(g(x)), -g(x + a * D))).min(1)).reshape(-1, 1))

        f0 = f(x) + np.matmul(np.transpose(w),abs(np.hstack((np.zeros_like(g(x)), -g(x))).min(1)).reshape(-1, 1))
        boolean = np.hstack((np.zeros_like(g(x)), g(x))).max(1).reshape(-1, 1) > 0
        dmaxg = np.zeros((len(boolean), 1))
        dmaxg0 = np.matmul(dg(x), D)
        dmaxg[boolean] = dmaxg0[boolean]
        df0 = np.matmul(np.transpose(df(x)), D) + np.matmul(np.transpose(w), dmaxg)
        phi_a = f0 + t * a * df0

        if f_a < phi_a:
            return a, w
        else:
            a = a * b
            counter += 1

def BFGS(W, dx, X, df):
    y = df(X) - df(X - dx)
    Q = np.matmul(np.matmul(dx.T, W), dx)
    if np.matmul(dx.T, y) >= 0.2 * Q:
        theta = 1
    else:
        theta = 0.8 * Q / (Q - np.matmul(dx.T, y))

    y = theta * y + (1 - theta) * np.matmul(W, dx)
    W = W + np.matmul(y, y.T) / np.matmul(y.T, dx) - np.matmul(np.matmul(W, dx), np.matmul(dx.T, W))
    return W

from myQP import myQP

[delx_lagrange, delmu_lagrange] = del_lagrangian(Xk, muk)

while np.linalg.norm(delx_lagrange) > eps_term and counter < 100:
    [sk, muk1] = myQP(Xk, W, funcx, ineq_constraint, dgx)
    alphak, w = linesearch(func, funcx, ineq_constraint, dgx, Xk, sk, muk1, w)
    Xk = Xk + alphak*sk
    W = BFGS(W, alphak*sk, Xk, funcx)
    [delx_lagrange, delmu_lagrange] = del_lagrangian(Xk, muk1)
    counter += 1

print(Xk)


[[1.06090661]
 [1.45664451]]


The final solution is $ x_1 = 1.06090661\ $ and $ x_2 = 1.45664451$